In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline 
from tensorflow.examples.tutorials.mnist import input_data


In [2]:
w1_initial=np.random.normal(size=(32*32*3,100)).astype(np.float32)
w2_initial=np.random.normal(size=(100,100)).astype(np.float32)
w3_initial=np.random.normal(size=(100,10)).astype(np.float32)

epsilon=1e-3


In [3]:
x_pre=tf.placeholder(tf.float32 , shape=[None,32,32,3])
x_ = tf.reshape(x_pre , shape=[-1,32*32*3])
y_=tf.placeholder(tf.float32 , shape=[None , 10])


In [4]:
train_img=np.load('./cifar_merge/train_img.npy')
train_lab=np.load('./cifar_merge/train_lab.npy')
test_img=np.load('./cifar_merge/test_img.npy')
test_lab=np.load('./cifar_merge/test_lab.npy')
val_img=np.load('./cifar_merge/val_img.npy')
val_lab=np.load('./cifar_merge/val_lab.npy')

print np.shape(val_img)
print np.shape(test_img)


(5000, 32, 32, 3)
(5000, 32, 32, 3)


In [5]:
with tf.device('/gpu:0'):
    def next_batch(batch_size , image , label):

        a=np.random.randint(np.shape(image)[0] -batch_size)
        batch_x = image[a:a+batch_size,:]
        batch_y= label[a:a+batch_size,:]
        return batch_x, batch_y


In [6]:
def batch_norm_wrapper(inputs , is_training , decay = 0.999):
    scale=tf.Variable(tf.ones([inputs.get_shape()[-1]]))
    beta =tf.Variable(tf.ones([inputs.get_shape()[-1]]))
    pop_mean = tf.Variable(tf.zeros([inputs.get_shape()[-1]]),trainable=False)
    pop_var  = tf.Variable(tf.zeros([inputs.get_shape()[-1]]),trainable=False)
    
    if is_training:
        batch_mean , batch_var = tf.nn.moments(inputs,[0])
        train_mean = tf.assign(pop_mean , pop_mean*decay+batch_mean*(1-decay))
        train_var  = tf.assign(pop_var  , pop_var*decay +batch_var*(1-decay))
        with tf.control_dependencies([train_mean , train_var]):
            return tf.nn.batch_normalization(inputs, batch_mean , batch_var , beta , scale , epsilon)
    else:
        return tf.nn.batch_normalization(inputs , pop_mean , pop_var , beta , scale , epsilon)
    

In [7]:
def build_graph(is_training ,x_,y_):


    w1 = tf.Variable(w1_initial)
    z1 = tf.matmul(x_,w1)
    bn1 = batch_norm_wrapper(z1 , is_training)
    l1 = tf.nn.sigmoid(bn1)

    #layer 2

    w2 = tf.Variable(w2_initial)
    z2 = tf.matmul(l1 , w2)
    bn2 = batch_norm_wrapper(z2 , is_training)
    l2 = tf.nn.sigmoid(bn2)

    w3 = tf.Variable(w3_initial)
    b3 = tf.Variable(tf.zeros([10]))
    y = tf.nn.softmax(tf.matmul(l2 , w3)+b3)

    cross_entropy = -tf.reduce_sum(y_*tf.log(y))
    train_step = tf.train.GradientDescentOptimizer(0.001).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.arg_max(y ,1) , tf.arg_max(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction , tf.float32))

    return (x_, y_) , train_step , accuracy , y ,tf.train.Saver()

In [8]:

(x,y_) , train_step , accuracy , y ,saver = build_graph(True,x_,y_)
sess=tf.Session()
init=tf.initialize_all_variables()
sess.run(init)
acc=[]


Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [9]:

for i in range(100):
    batch_xs , batch_ys = next_batch( 60 , train_img , train_lab)
    sess.run(accuracy , feed_dict={x_pre:batch_xs , y_ : batch_ys})
    if i%50 is 0:
        res = sess.run([accuracy],feed_dict={x_pre:test_img , y_ :test_lab})
        acc.append(res[0])
        saved_model = saver.save(sess, 'temp-bn-save1')
print saved_model
print "final_accuracy" , acc[-1]

temp-bn-save1
final_accuracy 0.1004


In [10]:
tf.reset_default_graph()

In [11]:
x_pre=tf.placeholder(tf.float32 , shape=[None,32,32,3])
x_ = tf.reshape(x_pre , shape=[-1,32*32*3])
y_=tf.placeholder(tf.float32 , shape=[None , 10])
(x,y_) , _ ,accuracy , y , saver = build_graph(True ,x_, y_)

prediction = [] 
correct =0

sess=tf.Session()
sess.run(tf.initialize_all_variables())
saver.restore(sess , saved_model)


Instructions for updating:
Use `tf.global_variables_initializer` instead.


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for temp-bn-save1
	 [[Node: save/RestoreV2_2 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save/Const_0, save/RestoreV2_2/tensor_names, save/RestoreV2_2/shape_and_slices)]]
	 [[Node: save/RestoreV2_9/_9 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_83_save/RestoreV2_9", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op u'save/RestoreV2_2', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/l

In [ ]:
    for i in range(100):
        pred , corr = sess.run([tf.arg_max(y ,1) , accuracy] , feed_dict = {x_pre: test_img , y_: test_lab})
        correct += corr 
        predictions.append(pred[0])
    print ("PREDICTIONS" , predictions)
    print ("ACCURACY" , correct/100)